In [1]:
# optimization of spacings between heliostats

In [2]:
import os
import math
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html

import scipy
from scipy.optimize import minimize_scalar
from scipy.optimize import minimize
from scipy.optimize import Bounds
from scipy import optimize

## Generation of layout

In [76]:
def findSpacings(xMin, xMax, xStep, isCentered, nMax):
    # possible positions
    x0 = 0. if isCentered else xStep/2;
    ansP = np.arange(x0, xMax, xStep)
    ansN = -np.arange(-(x0 - xStep), -xMin, xStep) 
    ans = np.concatenate((ansN, ansP))
    # select
    cmp = [(i, abs(v)) for (i, v) in enumerate(ans)]
    cmp = sorted(cmp, key=lambda x: x[1])
    nMax = min(nMax, len(cmp))
    ans = [ans[cmp[n][0]] for n in range(nMax)]
    # sort
    ans = np.sort(ans)
    return ans

In [81]:
def writeRow(letter, spacings, y, z, f):
    ans = ""
    for (i, x) in enumerate(spacings):
        ans += "{name},{sx:.3f} {sy:.3f} {sz:.3f},{sf:.3f}\n".format(
        name=letter + str(i + 1).zfill(2),
        sx=x,
        sy=y,
        sz=z,
        sf=f
    )
    return ans

In [145]:
def writeLayout(s):
    out = "Name,Position [m],Focus [m]\n"
    out += writeRow("A", spacings(-35.11, 27., s[0], True, 7), 46, 10, 46)
    out += writeRow("B", spacings(-42.5, 30.33, s[1], False, 8), 52, 14.5, 52)
    out += writeRow("C", spacings(-50.77, 15.83, s[2], True, 5), 58, 19.5, 58)
    #print(out)
    fileOut = open("heliostatsB.csv","w")
    fileOut.write(out)
    fileOut.close()

In [131]:
spacings(-50.77, 15.83, 8.9, True, 5)

array([-26.7, -17.8,  -8.9,   0. ,   8.9])

## Ray tracing

In [94]:
bat = "cmd /k {tonatiuh} -i={file}".format(
    tonatiuh = os.getcwd() + "/../../../installers/portable/bin/Tonatiuh-Application.exe",
    #tonatiuh = os.getcwd() + "/../../../bin/Tonatiuh-Application.exe",
    file="makeField.tnhpps"
)

def tnEta(x):
    # input
    writeLayout(x)
    
    # simulation
    os.system(bat)
    
    # output
    f = open("out.csv", "r")
    txt = f.read()
    f.close()
    return float(txt) 

In [96]:
eta = tnEta([6.5, 6.5, 6.5])
eta

0.28055716595207797

## Scalar

In [166]:
def tnFunc(x):
    return -tnEta([x, x, x])

In [168]:
res = minimize_scalar(tnFunc, method='bounded', bounds=(5., 10.), options={'disp':3,'maxiter':20,'xatol':0.01})

 
 Func-count     x          f(x)          Procedure
    1        6.90983    -0.280625        initial
    2        8.09017    -0.289792        golden
    3        8.81966    -0.292102        golden
    4         9.1124     -0.27674        parabolic
    5        8.54102    -0.290754        golden
    6        8.93148    -0.293681        golden
    7        9.00058    -0.278506        golden
    8        8.88877    -0.292408        golden
    9        8.95787    -0.291934        golden
   10        8.91516    -0.291954        golden
   11        8.94156    -0.292222        golden
   12        8.92524    -0.292683        golden
   13        8.93533    -0.292232        golden

Optimization terminated successfully;
The returned value satisfies the termination criteria
(using xtol =  0.01 )


In [169]:
res

     fun: -0.29368086389342885
 message: 'Solution found.'
    nfev: 13
  status: 0
 success: True
       x: 8.931476221966411

## Multivariate

In [175]:
scipy.optimize.show_options(solver="minimize", method="cobyla")

Minimize a scalar function of one or more variables using the
Constrained Optimization BY Linear Approximation (COBYLA) algorithm.

Options
-------
rhobeg : float
    Reasonable initial changes to the variables.
tol : float
    Final accuracy in the optimization (not precisely guaranteed).
    This is a lower bound on the size of the trust region.
disp : bool
    Set to True to print convergence messages. If False,
    `verbosity` is ignored as set to 0.
maxiter : int
    Maximum number of function evaluations.
catol : float
    Tolerance (absolute) for constraint violations


In [176]:
nFunc = 0
def tnFunc(x):
    global nFunc
    nFunc = nFunc + 1
    ans = tnEta(x)
    print('iteration {0:d}: {1:.3f}, {2:.3f}, {3:.3f}, f={4:.4f}'.format(nFunc, x[0], x[1], x[2], ans))
    return -ans

res = minimize(
    tnFunc, np.array([6.5, 6.5, 6.5]), constraints=[], tol=0.01,
    method='cobyla', options={'maxiter': 1000, 'disp': True, 'catol': 0.1}
)

iteration 1: 6.500, 6.500, 6.500, f=0.2758
iteration 2: 7.500, 6.500, 6.500, f=0.2783
iteration 3: 7.500, 7.500, 6.500, f=0.2856
iteration 4: 7.500, 7.500, 7.500, f=0.2877
iteration 5: 7.819, 8.408, 7.771, f=0.2905
iteration 6: 6.945, 8.874, 7.910, f=0.2865
iteration 7: 8.739, 8.517, 8.148, f=0.2924
iteration 8: 9.213, 8.871, 7.343, f=0.2754
iteration 9: 8.459, 8.496, 9.108, f=0.2945
iteration 10: 8.592, 7.506, 9.168, f=0.2853
iteration 11: 8.457, 9.466, 9.350, f=0.2800
iteration 12: 8.458, 8.981, 9.229, f=0.2917
iteration 13: 8.698, 8.479, 9.178, f=0.2915
iteration 14: 8.004, 8.296, 9.050, f=0.2914
iteration 15: 8.644, 8.357, 9.202, f=0.2914
iteration 16: 8.513, 8.525, 8.866, f=0.2918
iteration 17: 8.435, 8.604, 9.166, f=0.2926
iteration 18: 8.357, 8.423, 9.108, f=0.2919
iteration 19: 8.511, 8.470, 9.132, f=0.2936
iteration 20: 8.474, 8.512, 9.050, f=0.2937
iteration 21: 8.434, 8.476, 9.107, f=0.2915
iteration 22: 8.463, 8.524, 9.120, f=0.2924
iteration 23: 8.465, 8.497, 9.094, f=0.29

In [177]:
res

     fun: -0.2925167032252503
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 26
  status: 1
 success: True
       x: array([8.45179948, 8.48854137, 9.105979  ])

In [179]:
tnFunc(res.x)

iteration 28: 8.452, 8.489, 9.106, f=0.2923


-0.2922722265902916